This notebook demonstrates the working principle of our framework.

We would do the following things:

- Loading a progress learner model that has been trained on real human demonstrations.

- Using the learned model, demonstrate a simple demonstration on 2-D visualizer. This demo will demonstrate the result using a simple searching (*exploration only mode*) to find a good demonstration, and finish when the progress threshold attains a certain value.

First we load several needed libraries:

In [4]:
import os
import sys
import collections

# Import third party libraries
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline


# Import local libraries
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
a = os.path.join(module_path, "strands_qsr_lib\qsr_lib\src3")

sys.path.append(a)

import plotting
from rl import action_learner, action_learner_search, value_estimator, discrete_action_learner_search as dal
import progress_learner
import config
import project
# Need to add this import to load class
from project import Project
from rl import block_movement_env

A project used to store data for training, and other meta-data for each action (name *../slidearound_project.proj*

The model that has been learned for action **Slide Around** is stored at .

In the following code, we create a default setup of config. Some important variables in config file is:

- **progress_threshold**: Because we have learned a progress function for 

- **keep_branching**: 

- **branching**: 

In [ ]:
p = project.Project.load("../slidearound_project.proj")

c = config.Config()
tf.reset_default_graph()

global_step = tf.Variable(0, name="global_step", trainable=False)

sess =  tf.Session()

with tf.variable_scope("search") as scope:
    policy_estimator = action_learner_search.PolicyEstimator(c)

sess.run(tf.global_variables_initializer())

with tf.variable_scope("model") as scope:
    print('-------- Load progress model ---------')
    progress_estimator = progress_learner.EventProgressEstimator(is_training=False, name = p.name, config = c)  

# Print out all variables that would be restored
for variable in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model'):
    print (variable.name)

saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model'))
saver.restore(sess, '../progress_SlideAround.mod')